In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
from PIL import Image
import cv2
import numpy as np

filename = 'monkey-giraffe.mp4'
capture = cv2.VideoCapture(filename)
x_upper, image_shape = [], None
while True:
    success, image = capture.read()
    if not success:
        break
    image_shape = image.shape
    x_upper.append(image.reshape(-1))
capture.release()
x_upper = np.array(x_upper).T

In [ ]:
from timeit import default_timer as timer

from pydmd import DMD

now = timer()
dmd = DMD(svd_rank=-1, exact=False).fit(x_upper)
print(timer() - now)

In [ ]:
from numpy.linalg import norm

index = np.argmin(np.abs(np.log(dmd.eigs)))
dmd.plot_eigs()
print(F'index = {index}')
print(F'amplitude = {dmd.amplitudes[index]}')
print(F'eig = {dmd.eigs[index]}')
print(F'norm(mode) = {norm(dmd.modes[:, index], axis=0)}')

In [ ]:
reconstructed_data = np.outer(dmd.modes[:, index], dmd.dynamics[index])
background = np.round(reconstructed_data.T.real).astype(np.uint8, copy=False)
foreground = x_upper.T - background

In [ ]:
frame = 85
Image.fromarray(background[frame].reshape(*image_shape))

In [ ]:
Image.fromarray(foreground[frame].reshape(*image_shape))